In [ ]:
from common import *
use_hvplot()

In [ ]:
client = start_cluster(6, 8)
client

In [ ]:
year = 2021
water_year = f"wy{year}"
time=slice(f"{year -1}-10-01", f"{year}-07-31")

before_albedo = slice(f"{year -1}-10-01", f"{year}-03-14")
albedo_start = pd.to_datetime(f"{year}-03-15")
after_albedo = slice(f"{year}-03-15", f"{year}-08-01")

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(
    SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json'
)

### SMRF run 1

### Label for the run

In [ ]:
RUN_1 = 'SMRF - F06'

In [ ]:
smrf_run_1 = xr.open_mfdataset(
    (SNOBAL_DIR / water_year / 'erw/*/snow.nc').as_posix(),
    data_vars=['thickness'],
    parallel=True,
).sel(time=time)

butte_snobal_run_1 = smrf_run_1.sel(
    x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat
)
schofield_snobal_run_1 = smrf_run_1.sel(
    x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat
)
taylor_snobal_run_1 = smrf_run_1.sel(
    x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat
)

### SMRF run 2

### Label for the run

In [ ]:
RUN_2 = 'SMRF - F01'

In [ ]:
smrf_run_2 = xr.open_mfdataset(
    (SNOBAL_DIR / f"{water_year}_f01" / 'erw/*/snow.nc').as_posix(),
    data_vars=['thickness'],
    parallel=True,
).sel(time=time)

butte_snobal_run_2 = smrf_run_2.sel(
    x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat
)
schofield_snobal_run_2 = smrf_run_2.sel(
    x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat
)
taylor_snobal_run_2 = smrf_run_2.sel(
    x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat
)

## Compare across SMRF runs
### Snow Depth

In [ ]:
def compare_run(data, label):
    return data.thickness.coarsen(**COARSEN_OPTS).mean(). \
        mean(['x', 'y']).hvplot(label=label)

In [ ]:
compare_run(butte_snobal_run_1, RUN_1) \
    .opts(height=640, width=960, title='Butte Snow Depth') * \
compare_run(butte_snobal_run_2, RUN_2)

In [ ]:
compare_run(schofield_snobal_run_1, RUN_1) \
    .opts(height=640, width=960, title='Schofield Snow Depth') * \
compare_run(schofield_snobal_run_2, RUN_2)

In [ ]:
compare_run(taylor_snobal_run_1, RUN_1) \
    .opts(height=640, width=960, title='Taylor Snow Depth') * \
compare_run(taylor_snobal_run_2, RUN_2)

### Depth Difference 

In [ ]:
def mean_difference(a, b, time):
    return (
        a.thickness.sel(time=time) - 
        b.thickness.sel(time=time).reset_index('time', drop=True)
    ).coarsen(**COARSEN_OPTS).mean()

def run_difference(a, b, site):
    """
       Difference is: site data a - site data b 
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), dpi=150)
    
    mean_difference(a, b , before_albedo).plot(ax=ax1)
    ax1.set_title('Before albedo decay')
    
    mean_difference(a, b , after_albedo).plot(ax=ax2)
    ax2.set_title('With albedo decay')
    
    for ax in [ax1, ax2]:
        ax.set_ylabel('Snow Depth (m)')
    
    fig.suptitle(site.name + ' - Snow Depth Difference')

In [ ]:
run_difference(butte_snobal_run_1, butte_snobal_run_2, snotel_sites.Butte)
run_difference(schofield_snobal_run_1, schofield_snobal_run_2, snotel_sites.Schofield)
run_difference(taylor_snobal_run_1, taylor_snobal_run_2, snotel_sites.Taylor)

## ERW basin 
### Before Albedo decay

In [ ]:
time = before_albedo
title = "ERW Basin - Difference {0} to {1}: {time.start} - {time.stop}".format('{0}', '{1}', time=time)

In [ ]:
snotel_sites_plot = SnotelLocations()
snotel_sites_plot.load_from_json(
    SNOTEL_DIR / 'site-locations/snotel_sites.json'
)

In [ ]:
(
    smrf_run_1.thickness.sel(time=time).mean(dim=['time']) - 
    smrf_run_2.thickness.sel(time=time).mean(dim=['time'])
).hvplot(
    width=smrf_run_1.x.values.size, 
    height=smrf_run_1.y.values.size, 
    title=title.format(RUN_1, RUN_2)
) * \
hv.Points([
    [snotel_sites_plot.Butte.lon, snotel_sites_plot.Butte.lat],
    [snotel_sites_plot.Schofield.lon, snotel_sites_plot.Schofield.lat],
    [snotel_sites_plot.Taylor.lon, snotel_sites_plot.Taylor.lat]
]).opts(marker='o', size=10, color='orange')

### With Albedo Decay 

In [ ]:
time = after_albedo
title = "ERW Basin - Difference {0} to {1}: {time.start} - {time.stop}".format('{0}', '{1}', time=time)

In [ ]:
(
    smrf_run_1.thickness.sel(time=time).mean(dim=['time']) - 
    smrf_run_2.thickness.sel(time=time).mean(dim=['time'])
).hvplot(width=smrf_run_1.x.values.size, height=smrf_run_1.y.values.size, title=title.format(RUN_1, RUN_2)) * \
hv.Points([
    [snotel_sites_plot.Butte.lon, snotel_sites_plot.Butte.lat],
    [snotel_sites_plot.Schofield.lon, snotel_sites_plot.Schofield.lat],
    [snotel_sites_plot.Taylor.lon, snotel_sites_plot.Taylor.lat]
]).opts(marker='o', size=10, color='orange')